In [94]:
import ast
import astunparse
from typy import *

with open('test_nohints.py') as f:
    s = f.read()
sa = ast.parse(s)

print(astunparse.unparse(sa))



def test():
    a = {}
    return a

def return_str():
    a = A()
    if (a == 'test'):
        b = a
    return a.return_string()

class A():
    a = 'test'

    def return_string(self):
        return self.a
if (__name__ == '__main__'):
    print(return_str())



In [95]:
def get_type(context, astobj):
    #print(astobj.__class__.__name__)
    if astobj.__class__.__name__ == 'Constant':
        return [type(astobj.value).__name__]
    elif astobj.__class__.__name__ == 'List':
        return [astobj.__class__.__name__]
    elif astobj.__class__.__name__ == 'Dict':
        return [astobj.__class__.__name__]
    elif astobj.__class__.__name__ == 'Call':
        print('Call: ' + astobj.func.id)
        #get_type(context, astobj.func)
    elif astobj.__class__.__name__ == 'Name':
        print('Variable: ' + astobj.id)
    elif astobj.__class__.__name__ == 'Assign':
        get_type(context, astobj.value)
    elif astobj.__class__.__name__ == 'BinOp':
        lt = get_type(context, astobj.left)
        rt = get_type(context, astobj.right)
        if lt == rt:
            return lt
        elif lt == int and rt == float or lt == float and rt == int:
            return float
    else:
        print('Not implemented: ' + astobj.__class__.__name__)

print(get_type({'class':'A'}, sa.body[0].body[0].value))
print(get_type_candidate({'class':'A'}, sa.body[0].body[0].value))

['Dict']
['Dict']


In [83]:
def explore_body(context, astobj, nest=0):
    for i in range(nest):
        print('    ', end='')
    print(astobj.__class__.__name__)
    if astobj.__class__.__name__ == 'Assign':
        print(get_type(context, astobj.value))
    for b in getattr(astobj, 'body', []):
        if astobj.__class__.__name__ == 'FunctionDef':
            context['function'][astobj.name] = {
                'name': astobj.name,
                'returns': astobj.returns
            }
        if astobj.__class__.__name__ == 'ClassDef':
            context['class'][astobj.name] = {}
    for b in getattr(astobj, 'body', []):
        explore_body(context, b, nest + 1)
    return context
        
explore_body({'function':{}, 'class':{}}, sa)

Module
    FunctionDef
        Return
    FunctionDef
        Assign
Call: A
None
        If
            Assign
Variable: a
None
        Return
    ClassDef
        Assign
['str']
        FunctionDef
            Return
    If
        Expr


{'function': {'test': {'name': 'test', 'returns': None},
  'return_str': {'name': 'return_str', 'returns': None},
  'return_string': {'name': 'return_string', 'returns': None}},
 'class': {'A': {}}}